In [104]:
from preprocessing import *
from sklearn.model_selection import KFold
import argparse
from model import *
from train import test
import torch.optim as optim
import pandas as pd

from MatrixVectorizer import *
import networkx as nx
from typing import Union


In [121]:
# load csvs as numpy
lr_data_path = '../data/lr_train.csv'
hr_data_path = '../data/hr_train.csv'

lr_train_data = pd.read_csv(lr_data_path, delimiter=',').to_numpy()
hr_train_data = pd.read_csv(hr_data_path, delimiter=',').to_numpy()
lr_train_data[lr_train_data < 0] = 0
np.nan_to_num(lr_train_data, copy=False)

hr_train_data[hr_train_data < 0] = 0
np.nan_to_num(hr_train_data, copy=False)

# map the anti-vectorize function to each row of the lr_train_data

lr_train_data_vectorized = np.array([MatrixVectorizer.anti_vectorize(row, 160) for row in lr_train_data])
hr_train_data_vectorized = np.array([MatrixVectorizer.anti_vectorize(row, 268) for row in hr_train_data])

num_samples = hr_train_data_vectorized.shape[0]
num_samples_list = range(num_samples)
sample_to_index = dict(zip(num_samples_list, hr_train_data_vectorized))

split = int(num_samples * 0.8)

In [106]:

subjects_adj,subjects_labels = lr_train_data_vectorized[:split], hr_train_data_vectorized[:split]

held_out_subjects_adj,held_out_subjects_labels = lr_train_data_vectorized[split:], hr_train_data_vectorized[split:]

In [107]:
num_splt = 3
epochs = 10
lr = 0.00005
lmbda = 25
lr_dim = 160
hr_dim = 320
hidden_dim = 320
padding = 26
dropout = 0.1
args = argparse.Namespace()
args.epochs = epochs
args.lr = lr
args.lmbda = lmbda
args.lr_dim = lr_dim
args.hr_dim = hr_dim
args.hidden_dim = hidden_dim
args.padding = padding
args.p = dropout


In [108]:
cv = KFold(n_splits=3, random_state=42, shuffle=True)

In [109]:
ks = [0.7, 0.5]
model = GSRNet(ks, args)

In [136]:
class TopologicalMeasures:
    def __init__(self,graph:Union[np.ndarray,torch.Tensor]):
        if isinstance(graph,np.ndarray):
            self.graph = nx.Graph(graph)
        elif isinstance(graph,torch.Tensor):
            graph_numpy = graph.cpu().detach().numpy()
            self.graph = nx.Graph(graph_numpy)

    def compute_measures(self, number = 0):
        measures = {}
        measures['degree'] = torch.FloatTensor(list(dict(self.graph.degree()).values()))
        # measures['clustering'] = torch.FloatTensor(list(nx.clustering(self.graph).values())) # removed due to slow computation
        measures['closeness'] = torch.FloatTensor(list(nx.closeness_centrality(self.graph).values()))
        # measures['betweenness'] = torch.FloatTensor(list(nx.betweenness_centrality(self.graph).values())) # removed due to slow computation
        measures['pagerank'] = torch.FloatTensor(list(nx.pagerank(self.graph).values()))
        measures['eigenvector'] = torch.FloatTensor(list(nx.eigenvector_centrality(self.graph).values()))
        return measures

def precompute_topological_measures(hr_train_data_vectorized:np.ndarray):
    index_to_measure = {}
    for index, graph in enumerate(hr_train_data_vectorized):
        index_to_measure[index] = TopologicalMeasures(graph).compute_measures()
        if index % 10 == 0:
            print(f'Computed measures for {index} graphs')
    return index_to_measure


def compute_topological_MAE_loss(graph1,graph2:Union[np.ndarray,torch.Tensor], precomputed_g1: False):
    if precomputed_g1:
        measures1 = graph1
    else:
        measures1 = TopologicalMeasures(graph1).compute_measures()
    measures2 = TopologicalMeasures(graph2).compute_measures()
    loss = 0
    # compute MAE for each measure

    for measure in measures1:
        loss += F.l1_loss(measures1[measure], measures2[measure])
    loss = loss/len(measures1)

    return loss


In [132]:
# precompute topological measures for hr_train_data_vectorized
precomputed_measures = precompute_topological_measures(hr_train_data_vectorized)

Computed measures for 0 graphs
Computed measures for 10 graphs
Computed measures for 20 graphs
Computed measures for 30 graphs
Computed measures for 40 graphs
Computed measures for 50 graphs
Computed measures for 60 graphs
Computed measures for 70 graphs
Computed measures for 80 graphs
Computed measures for 90 graphs
Computed measures for 100 graphs
Computed measures for 110 graphs
Computed measures for 120 graphs
Computed measures for 130 graphs
Computed measures for 140 graphs
Computed measures for 150 graphs
Computed measures for 160 graphs


In [147]:
import time
criterion = nn.L1Loss()

def train(model, optimizer, subjects_adj,subjects_labels, args): 
  #, subjects_adj_test, subjects_ground_truth_test):
  
  all_epochs_loss = []
  no_epochs = args.epochs

  for epoch in range(no_epochs):
    epoch_loss = []
    epoch_error = []
    epoch_topo = []

    model.train()
    for index,(lr,hr) in enumerate(zip(subjects_adj,subjects_labels)):      
      lr = torch.from_numpy(lr).type(torch.FloatTensor)
      hr = torch.from_numpy(hr).type(torch.FloatTensor)

      model_outputs,net_outs,start_gcn_outs,layer_outs = model(lr)
      model_outputs  = unpad(model_outputs, args.padding)
      
      # weights = unpad(model.layer.weights, args.padding)

      padded_hr = pad_HR_adj(hr,args.padding)
      eig_val_hr, U_hr = torch.linalg.eigh(padded_hr, UPLO='U')

      # loss = criterion(net_outs, start_gcn_outs) + criterion(model.layer.weights,U_hr) + args.lmbda * criterion(model_outputs, hr) 
      # loss = criterion(model_outputs, hr) 
      loss = args.lmbda * criterion(net_outs, start_gcn_outs) + criterion(model.layer.weights,U_hr) + criterion(model_outputs, hr) 
      start_time = time.time()

      # topo = compute_topological_MAE_loss(hr, model_outputs, precomputed_g1 = False)
      topo = compute_topological_MAE_loss(precomputed_measures[index], model_outputs, precomputed_g1 = True)
      # print("Time to compute topo: ", time.time() - start_time)
      # print(topo.item())

      error = criterion(model_outputs, hr)
      
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      epoch_loss.append(loss.item())
      epoch_error.append(error.item())
      epoch_topo.append(topo.item())
      
  
    model.eval()
    # print("Epoch: ",epoch+1, "Loss: ", np.mean(epoch_loss), "Error: ", np.mean(epoch_error))
    print("Epoch: ",epoch+1, "Loss: ", np.mean(epoch_loss), "Error: ", np.mean(epoch_error), "Topo: ", np.mean(epoch_topo))
    # test(model, held_out_subjects_adj, held_out_subjects_labels, args)
    # test(model, subjects_adj_test, subjects_ground_truth_test, args)
    all_epochs_loss.append(np.mean(epoch_loss))

In [112]:
# # print(model)
# optimizer = optim.Adam(model.parameters(), lr=args.lr)
# # optimizer = optim.SGD(model.parameters(), lr=args.lr)

# for train_index, test_index in cv.split(subjects_adj):
#     subjects_adj_train = subjects_adj[train_index]  # Get training data 
#     subjects_adj_test = subjects_adj[test_index]   # Get testing data 
#     subjects_ground_truth_train = subjects_labels[train_index]
#     subjects_ground_truth_test = subjects_labels[test_index]

#     train(model, optimizer, subjects_adj_train, subjects_ground_truth_train, args, subjects_adj_test, subjects_ground_truth_test)
    
#     print('Held out test score:')
#     test(model, held_out_subjects_adj, held_out_subjects_labels, args)
#     print('------------------------------')

# Final Model & Kaggle Submission

In [148]:
#final train
final_model = GSRNet(ks, args)
optimizer = optim.Adam(final_model.parameters(), lr=args.lr)

train(final_model, optimizer, lr_train_data_vectorized, hr_train_data_vectorized, args)

Epoch:  1 Loss:  0.5797495531464765 Error:  0.23944988998467337 Topo:  13.371520245146609
Epoch:  2 Loss:  0.381369464947078 Error:  0.21050367169751377 Topo:  12.77267477755061
Epoch:  3 Loss:  0.3578568504242126 Error:  0.1977207723135006 Topo:  12.793938816664461


KeyboardInterrupt: 

In [ ]:
#Generate submission 

# load csvs as numpy
test_lr_data_path = '../data/lr_test.csv'

# lr_test_data = np.loadtxt(test_lr_data_path, delimiter=',')
lr_test_data = pd.read_csv(test_lr_data_path, delimiter=',').to_numpy()
print(lr_test_data.shape)
lr_test_data[lr_test_data < 0] = 0
np.nan_to_num(lr_test_data, copy=False)


# map the anti-vectorize function to each row of the lr_train_data

lr_test_data_vectorized = np.array([MatrixVectorizer.anti_vectorize(row, 160) for row in lr_test_data])
print(lr_test_data_vectorized.shape)

(112, 12720)
(112, 160, 160)


In [ ]:
final_model.eval()
preds = []
for lr in lr_test_data_vectorized:      
  lr = torch.from_numpy(lr).type(torch.FloatTensor)
  
  model_outputs, _, _, _ = final_model(lr)
  model_outputs  = unpad(model_outputs, args.padding)
  preds.append(MatrixVectorizer.vectorize(model_outputs.detach().numpy()))

print(len(preds), preds[0].shape)
r = np.hstack(preds)
print(r.shape)
meltedDF = r.flatten()

112 (35778,)
(4007136,)


In [ ]:
n = meltedDF.shape[0]
df = pd.DataFrame({'ID': np.arange(1, n+1),
                   'Predicted': meltedDF})
df.to_csv('submission.csv', index=False)